In [ ]:
# Standard Libraries
import re
import math
import warnings

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

# Statistical analysis
from scipy.stats import boxcox, skew
from statsmodels.stats.outliers_influence import variance_inflation_factor

# MLflow
import mlflow
import mlflow.sklearn

# Suppress warnings
warnings.filterwarnings('ignore')

## Data Overview

In [ ]:
df = pd.read_csv('weather_prediction_dataset.csv')
df.head()

In [ ]:
df.columns

In [ ]:
cities = ['BASEL', 'BUDAPEST', 'DE_BILT', 'DRESDEN', 'DUSSELDORF', 'HEATHROW', 'KASSEL', 
          'LJUBLJANA', 'MAASTRICHT', 'MALMO', 'MONTELIMAR', 'MUENCHEN', 'OSLO', 'PERPIGNAN', 
          'ROMA', 'SONNBLICK', 'STOCKHOLM', 'TOURS']
print("\nCities:")
print(sorted(cities))


In [ ]:
city_var_pairs = [col for col in df.columns if any(col.startswith(city) for city in cities)]
print("\nCity Variable pairs:")
print(city_var_pairs)

In [ ]:
variables = set()
city_variable_dict = {}

# Extract weather variables based on city names
for city in cities:
    found_variables = [col.split(city + "_")[1] for col in city_var_pairs if col.startswith(city + "_")]
    variables.update(set(found_variables))
    city_variable_dict[city] = found_variables

# Convert variables set to sorted list
variable_list = sorted(variables)
print("\nWeather Variables:")
print(variable_list)

In [ ]:
# Cities are rows and variables are columns, populated with 0/1
data_matrix = pd.DataFrame(0, index=cities, columns=variable_list)

for city, var_list in city_variable_dict.items():
    for var in var_list:
        data_matrix.loc[city, var] = 1

plt.figure(figsize=(12, 8))
sns.heatmap(data_matrix, annot=True, cmap="Blues", cbar=False, linewidths=0.5)
plt.title('City vs Weather Variables')
plt.xlabel('Weather Variables')
plt.ylabel('Cities')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

As there are 18 cities, building a generalizable model requires a more compact and scalable feature space.

Efficient Feature Space: The number of features remains constant, regardless of the number of cities, simplifying the model and avoiding unnecessary complexity.

Improved Generalization: The model generalizes more effectively since the feature structure stays consistent, with the city treated as just one additional feature.

Scalability: This approach scales efficiently when handling many cities or expanding the model to include new ones.

Categorical Encoding: The city column can easily be encoded as a categorical feature (e.g., using one-hot encoding or embeddings) without significantly increasing the feature space.

Compact Feature Space: Instead of generating a separate column for each city-variable combination, you only need one set of columns for weather variables, which significantly reduces dimensionality.

Optimized for Model Training: By minimizing dimensionality, this approach mitigates the curse of dimensionality, making the dataset more efficient for training machine learning models. The city can be encoded using techniques like one-hot encoding or embeddings.

Flexible and Scalable: This structure is highly adaptable, making it easier to add more cities or extend the model to a longer time range.

### Data Reshaping

In [ ]:
cities = ['BASEL', 'BUDAPEST', 'DE_BILT', 'DRESDEN', 'DUSSELDORF', 'HEATHROW', 'KASSEL',
          'LJUBLJANA', 'MAASTRICHT', 'MALMO', 'MONTELIMAR', 'MUENCHEN', 'OSLO', 'PERPIGNAN',
          'ROMA', 'SONNBLICK', 'STOCKHOLM', 'TOURS']

weather_variables = ['cloud_cover', 'global_radiation', 'humidity', 'precipitation', 'pressure', 'sunshine', 'temp_max', 'temp_mean', 'temp_min', 'wind_gust', 'wind_speed']

city_var_pairs = [col for col in df.columns if any(col.startswith(city) for city in cities)]

In [ ]:
def extract_city_and_variable(col_name):
    for city in cities:
        if col_name.startswith(city):
            return city, col_name[len(city)+1:]  # Extract city and variable
    return None, None

In [ ]:
# Reshape the dataframe to long format
long_format_df = pd.melt(
    df,
    id_vars=['DATE', 'MONTH'],  # Keep DATE and MONTH columns as they are
    value_vars=city_var_pairs,  # Use city-variable pairs
    var_name='city_variable',  # Temporary column for city-variable combined names
    value_name='value'  # Column for the values
)

# Split the 'city_variable' into 'city' and 'variable' using the custom function
long_format_df[['city', 'variable']] = long_format_df['city_variable'].apply(lambda x: extract_city_and_variable(x)).apply(pd.Series)

# Drop the 'city_variable' column since it's no longer needed
long_format_df.drop(columns=['city_variable'], inplace=True)

# Pivot the table to get the final reshaped format
reshaped_df = long_format_df.pivot_table(index=['DATE', 'MONTH', 'city'], columns='variable', values='value').reset_index()

reshaped_df.to_csv('reshaped_weather_prediction_dataset.csv', index=False)

## Data Overview (after reshaping)

In [ ]:
reshaped_df = pd.read_csv('reshaped_weather_prediction_dataset.csv')
reshaped_df.head()

In [ ]:
reshaped_df.describe()

In [ ]:
reshaped_df.info()

In [ ]:
reshaped_df.dtypes

### Confirming dataset is time-based. Yes, the dataset is time-based as the dates are unique and ordered chronologically  (monotonically increasing) and there are no missing dates

In [ ]:
reshaped_df['DATE'] = pd.to_datetime(reshaped_df['DATE'], format='%Y%m%d')

# Check for unique dates and their order
unique_dates = reshaped_df['DATE'].nunique()
total_rows = len(reshaped_df)
date_order_correct = reshaped_df['DATE'].is_monotonic_increasing

# Check for any missing or irregular time intervals
date_range = pd.date_range(start=reshaped_df['DATE'].min(), end=reshaped_df['DATE'].max())
missing_dates = date_range.difference(reshaped_df['DATE'].unique())

unique_dates, total_rows, date_order_correct, len(missing_dates), missing_dates[:10]

## Missing Data Analysis

In [ ]:
missing_data = reshaped_df.isnull().sum()
missing_data = missing_data[missing_data > 0]
missing_data

In [ ]:
missing_data_proportion = reshaped_df.isnull().mean() * 100
missing_data_proportion = missing_data_proportion[missing_data_proportion > 0]
missing_data_proportion = missing_data_proportion.sort_values(ascending=False)
missing_data_proportion_formatted = missing_data_proportion.apply(lambda x: f"{x:.2f}%")
missing_data_proportion_formatted

Variables with Low Missing Data (<10%): 

precipitation and temp_min are the least affected by missing data and be left as is, as the low percentage of missing data is unlikely to significantly impact model training. 

Variables with Moderate Missing Data (10-30%): 
cloud_cover, global_radiation, humidity, pressure, sunshine, and wind_speed fall are moderately affected by missing data.As the dataset has been confirmed to be time-based, time-based imputation is an effective approach.

Forward-fill: The missing values for a variable are filled using the most recent available value (i.e., carry the last known value forward).

Backward-fill: If there are missing values at the beginning of a series, backward-fill can be used to propagate future values backward.

Variables with High Missing Data (>30%): 

wind_gust has 61.11% missing data, making it unreliable for use in model training.Given the high percentage of missing data, the variable is dropped from analysis entirely, as attempting to impute such a large portion may introduce bias or noise.

In [ ]:
# Drop 'wind_gust' due to high percentage of missing data
reshaped_df = reshaped_df.drop(columns=['wind_gust'])

mean_imputer = SimpleImputer(strategy='mean')
constant_imputer = SimpleImputer(strategy='constant', fill_value=0)
# Using mean imputation for 'temp_min'
reshaped_df['temp_min'] = mean_imputer.fit_transform(reshaped_df[['temp_min']])

# Using constant imputation (0) for 'precipitation'
reshaped_df['precipitation'] = constant_imputer.fit_transform(reshaped_df[['precipitation']])

# Variables with moderate missing data to be imputed
variables_to_impute = ['cloud_cover', 'global_radiation', 'humidity', 'pressure', 'sunshine', 'wind_speed']

# Apply forward-fill to impute missing values
reshaped_df[variables_to_impute] = reshaped_df[variables_to_impute].ffill()

# Apply backward-fill to handle any remaining missing values at the start of the series
reshaped_df[variables_to_impute] = reshaped_df[variables_to_impute].bfill()

In [ ]:
reshaped_df.to_csv('reshaped_weather_prediction_dataset.csv', index=False)
reshaped_df

In [ ]:
missing_data = reshaped_df.isnull().sum()
missing_data = missing_data[missing_data > 0]
missing_data

In [ ]:
reshaped_df['year'] = reshaped_df['DATE'].dt.year
reshaped_df['month'] = reshaped_df['DATE'].dt.month
reshaped_df['day'] = reshaped_df['DATE'].dt.day
reshaped_df.drop(columns=['DATE', 'MONTH'], inplace=True)
reshaped_df.head()

In [ ]:
reshaped_df.describe()

## Feature Distribution and Univariate Analysis

In [ ]:
categorical_features = reshaped_df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_features = reshaped_df.select_dtypes(include=['number']).columns.tolist()

print("Categorical Features:")
print(categorical_features)

print("\nDistinct values for each categorical feature:")
for feature in categorical_features:
    distinct_values = reshaped_df[feature].nunique()
    unique_values = reshaped_df[feature].unique()
    print(f"{feature}: {distinct_values} unique values")
    print(f"Values: {unique_values}")

print("\nNumerical Features:")
print(numerical_features)

In [ ]:
numerical_features = ['cloud_cover', 'global_radiation', 'humidity', 'precipitation', 
                      'pressure', 'sunshine', 'temp_max', 'temp_mean', 'temp_min', 
                      'wind_speed']

num_features = len(numerical_features)
cols = 3
rows = math.ceil(num_features / cols)

fig, axs = plt.subplots(rows, cols, figsize=(18, 6 * rows))
axs = axs.flatten()
 
for i, feature in enumerate(numerical_features):
    sns.histplot(reshaped_df[feature], bins=20, kde=True, ax=axs[i])
    axs[i].set_title(f'{feature.capitalize()} Distribution')
    axs[i].set_xlabel(feature.capitalize())
    axs[i].set_ylabel('Frequency')

for j in range(i + 1, len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()

### Shape Analysis (Skewness)

In [ ]:
numerical_features = ['cloud_cover', 'global_radiation', 'humidity', 'precipitation', 
                      'pressure', 'sunshine', 'temp_max', 'temp_mean', 'temp_min', 
                      'wind_speed']
skewness = reshaped_df[numerical_features].skew()
print("Skewness of features:\n", skewness)

Highly Skewed Features:
	•	cloud_cover, precipitation, pressure, and wind_speed are highly skewed, which could impact the performance of regression models like Linear Regression. These should be the focus for transformation.

Moderate Skewness:
	•	global_radiation, humidity, sunshine, and temp_min may still benefit from some transformation, but their skewness is less extreme.

In [ ]:
import numpy as np
from scipy.stats import boxcox

# Cloud Cover: Reflection + Log
reshaped_df['cloud_cover'] = np.log1p(-reshaped_df['cloud_cover'] + np.max(reshaped_df['cloud_cover']) + 1)

# Global Radiation: Box-Cox
reshaped_df['global_radiation'], _ = boxcox(reshaped_df['global_radiation'] + 1)

# Humidity: Box-Cox
reshaped_df['humidity'], _ = boxcox(reshaped_df['humidity'] + 1)

# Precipitation: Box-Cox
reshaped_df['precipitation'], _ = boxcox(reshaped_df['precipitation'] + 1)

# Pressure: Outlier Removal + Reflection + Log
Q1 = reshaped_df['pressure'].quantile(0.25)
Q3 = reshaped_df['pressure'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
reshaped_df = reshaped_df[(reshaped_df['pressure'] >= lower_bound) & (reshaped_df['pressure'] <= upper_bound)]
reshaped_df['pressure'] = np.log1p(-reshaped_df['pressure'] + np.max(reshaped_df['pressure']) + 1)

# Wind Speed: Log
reshaped_df['wind_speed'] = np.log1p(reshaped_df['wind_speed'])

new_skewness = reshaped_df[numerical_features].skew()
print("Skewness after transformation:\n", new_skewness)

The cloud_cover, global_radiation, humidity, pressure, and wind_speed features are now nearly perfectly normalized.

The temperature features are still slightly negatively skewed, but this is acceptable for most modeling scenarios.

The precipitation and sunshine features still have some moderate skewness, but these values are much improved and should not cause significant issues in most models.

In [ ]:
cols = 3 
num_features = len(numerical_features)
rows = (num_features // cols) + 1 if num_features % cols != 0 else num_features // cols
plt.figure(figsize=(16, 4 * rows))

for i, feature in enumerate(numerical_features, 1):
    plt.subplot(rows, cols, i)
    sns.kdeplot(reshaped_df[feature], shade=True)
    plt.title(f'Distribution of {feature} (KDE)')
    plt.xlabel(feature)
    plt.ylabel('Density')

plt.tight_layout()
plt.show()

### Outlier Analysis

In [ ]:
plt.figure(figsize=(16, 12))
for i, feature in enumerate(['cloud_cover', 'global_radiation', 'humidity', 
                             'precipitation', 'pressure', 'sunshine', 
                             'temp_max', 'temp_mean', 'temp_min', 'wind_speed'], 1):
    plt.subplot(5, 2, i)
    sns.boxplot(x=reshaped_df[feature])
    plt.title(f'Boxplot of {feature}')
plt.tight_layout()
plt.show()

In [ ]:
def calculate_iqr_bounds(df, column):
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # Calculate the outlier bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    return lower_bound, upper_bound

for feature in ['cloud_cover', 'global_radiation', 'humidity', 'precipitation', 
                'pressure', 'sunshine', 'temp_max', 'temp_mean', 'temp_min', 'wind_speed']:
    lower, upper = calculate_iqr_bounds(reshaped_df, feature)
    print(f"{feature} has bounds - Lower Bound: {lower}, Upper Bound: {upper}")

No outlier capping or removal required:

Cloud cover usually ranges from 0 (no cloud cover) to around 8 (completely overcast). The outliers detected here might not represent erroneous data but could indicate low cloud cover conditions. Since the upper bound is 2.79, extreme cloud cover values (close to fully overcast) are not flagged as outliers.

Humidity is typically expressed as a percentage (0%–100%). Depending on the transformation applied, values near 0.13 may represent very low humidity levels, which could occur in extremely dry climates. Upper values may represent high-humidity conditions, potentially caused by local weather.

Outlier capping required:
Temperature Features (temp_max, temp_mean, temp_min): These features have many outliers, but they may be valid extremes that represent important weather events and are extreme value capped as they are part of normal seasonal variation or extreme weather.

Wind speed typically follows a log-normal distribution, with most values clustering near the lower end and some extreme wind events. Outliers might represent strong winds or storms.

Outlier removal:
Sunshine is typically measured in hours per day. Negative values don’t make sense in this context and could represent erroneous entries.

In [ ]:
def cap_outliers(df, column, lower_bound, upper_bound):
    df[column] = np.clip(df[column], lower_bound, upper_bound)
    return df

capping_features = ['temp_max', 'temp_mean', 'temp_min', 'wind_speed']
bounds = {
    'temp_max': (-12.45, 42.35),
    'temp_mean': (-12.95, 34.65),
    'temp_min': (-13.1, 26.9),
    'wind_speed': (0.32, 2.56)
}

for feature in capping_features:
    lower_bound, upper_bound = bounds[feature]
    reshaped_df = cap_outliers(reshaped_df, feature, lower_bound, upper_bound)
    
reshaped_df = reshaped_df[reshaped_df['sunshine'] >= 0]

lower_bound_pressure = 0.693
upper_bound_pressure = 0.714
reshaped_df = cap_outliers(reshaped_df, 'pressure', lower_bound_pressure, upper_bound_pressure)

 # No outliers but keeping within these bounds for consistency
bounds_other = {
    'cloud_cover': (0.55, 2.79),
    'humidity': (0.13, 3.67),
    'global_radiation': (-0.64, 2.27)
}

for feature in ['cloud_cover', 'humidity', 'global_radiation']:
    lower_bound, upper_bound = bounds_other[feature]
    reshaped_df = cap_outliers(reshaped_df, feature, lower_bound, upper_bound)

In [ ]:
reshaped_df[['cloud_cover', 'global_radiation', 'humidity', 'precipitation', 
                'pressure', 'sunshine', 'temp_max', 'temp_mean', 'temp_min', 'wind_speed']].describe()

Transformation Effectiveness: The transformations and outlier handling techniques have ensured that all features fall within realistic, well-distributed ranges. The skewness and outlier issues have been addressed while retaining important weather patterns.

## Time Series Data Visualization

In [ ]:
temp_pivot = reshaped_df.pivot_table(values='temp_mean', index='year', columns='month', aggfunc='mean')

plt.figure(figsize=(10, 6))
sns.heatmap(temp_pivot, annot=True, fmt=".1f", cmap='coolwarm')
plt.title('Average Temperature by Year and Month')
plt.show()

In [ ]:
reshaped_df['datetime'] = pd.to_datetime(reshaped_df[['year', 'month', 'day']])

# Create a dropdown widget for selecting the city
city_dropdown = widgets.Dropdown(
    options=reshaped_df['city'].unique(),
    value=reshaped_df['city'].unique()[0],
    description='City:',
)

# List of features for visualization
features = ['sunshine', 'temp_max', 'temp_mean', 'temp_min']

def display_plots(city):
    city_data = reshaped_df[reshaped_df['city'] == city]
    
    for feature in features:
        fig = px.line(city_data, x='datetime', y=feature, title=f'{feature.replace("_", " ").title()} Over Time for {city}')
        fig.update_layout(xaxis_title='Time', yaxis_title=feature.replace("_", " ").title(), xaxis_rangeslider_visible=True)
        fig.show()

widgets.interact(display_plots, city=city_dropdown)

## Multivariate Analysis

### Correlation analysis

In [ ]:
numerical_features = ['cloud_cover', 'global_radiation', 'humidity', 'precipitation', 
                      'pressure', 'sunshine', 'temp_max', 'temp_min', 'wind_speed']
numerical_df = reshaped_df[numerical_features + ['temp_mean']]

# Compute Pearson and Spearman correlations with the target (temp_mean)
pearson_corr_with_target = numerical_df.corr(method='pearson')['temp_mean'].drop('temp_mean')
spearman_corr_with_target = numerical_df.corr(method='spearman')['temp_mean'].drop('temp_mean')

print("Pearson Correlation with Target (temp_mean):")
print(pearson_corr_with_target)

print("\nSpearman Correlation with Target (temp_mean):")
print(spearman_corr_with_target)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Pearson Correlation heatmap
sns.heatmap(pearson_corr_with_target.to_frame(), annot=True, cmap='coolwarm', fmt='.2f', ax=axes[0], cbar=False)
axes[0].set_title('Pearson Correlation with temp_mean')
axes[0].set_xticklabels(['temp_mean'], rotation=45, ha='right')
axes[0].set_yticklabels(pearson_corr_with_target.index, rotation=0)

# Plot Spearman Correlation heatmap
sns.heatmap(spearman_corr_with_target.to_frame(), annot=True, cmap='coolwarm', fmt='.2f', ax=axes[1], cbar=False)
axes[1].set_title('Spearman Correlation with temp_mean')
axes[1].set_xticklabels(['temp_mean'], rotation=45, ha='right')
axes[1].set_yticklabels(spearman_corr_with_target.index, rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
numerical_features = ['cloud_cover', 'global_radiation', 'humidity', 'precipitation', 
                      'pressure', 'sunshine', 'temp_max', 'temp_mean', 'temp_min', 
                      'wind_speed']
numerical_df = reshaped_df[numerical_features]
pearson_corr_matrix = numerical_df.corr(method='pearson')
spearman_corr_matrix = numerical_df.corr(method='spearman')

print("Pearson Correlation Matrix:")
print(pearson_corr_matrix)

print("\nSpearman Correlation Matrix:")
print(spearman_corr_matrix)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Pearson Correlation Heatmap
sns.heatmap(pearson_corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', ax=axes[0])
axes[0].set_title('Pearson Correlation Matrix')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, horizontalalignment='right')
axes[0].set_yticklabels(axes[0].get_yticklabels(), rotation=45)

# Spearman Correlation Heatmap
sns.heatmap(spearman_corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', ax=axes[1])
axes[1].set_title('Spearman Correlation Matrix')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, horizontalalignment='right')
axes[1].set_yticklabels(axes[1].get_yticklabels(), rotation=45)

plt.tight_layout()
plt.show()

Pearson Correlation Matrix Analysis (Linear Relationships)

cloud_cover and sunshine: Strong negative correlation which makes sense as more cloud cover generally means less sunshine.
sunshine and global_radiation : Sunshine hours are strongly related to global radiation, as expected in weather data.
humidity and temp_max: Higher temperatures are often associated with lower humidity, which is typical in dry, hot weather conditions.
Humidity shows consistent negative correlations with temperature (temp_max and temp_mean) and global radiation, which is typical in dry, sunny weather.

Spearman Correlation Matrix Analysis (Rank-Based Relationships)

cloud_cover and sunshine: The strong rank-based relationship here implies that more cloud cover is consistently associated with less sunshine.
temp_max and global_radiation (0.66): The rank correlation is slightly higher than the Pearson correlation, indicating a strong, possibly non-linear relationship.



### Multicollinearity Checks (VIF)

In [ ]:
# Calculate VIF for each feature
X = reshaped_df[numerical_features]  # Numerical features matrix
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

VIF is a measure of how much the variance of a regression coefficient is inflated due to multicollinearity among the independent variables.

VIF > 10: Indicates high multicollinearity
VIF > 5: Indicates moderate multicollinearity

Temp_max and Temp_mean are two temperature-related features that are highly collinear, likely due to their close relationship.

### Feature Engineering and Selection

In [ ]:
reshaped_df['temp_range'] = reshaped_df['temp_max'] - reshaped_df['temp_min']
reshaped_df.drop(['temp_max', 'temp_min'], axis=1, inplace=True)

In [ ]:
reshaped_df.drop(['pressure', 'cloud_cover'], axis=1, inplace=True)
reshaped_df['humidity'] = np.log1p(reshaped_df['humidity'])
reshaped_df['global_radiation'] = np.log1p(reshaped_df['global_radiation'])

In [ ]:
updated_numerical_features = ['global_radiation', 'humidity', 
                              'precipitation', 'sunshine', 
                              'wind_speed', 'temp_range', 'temp_mean']

X = reshaped_df[updated_numerical_features]
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

## Multivariate Analysis

### Feature Importance

In [ ]:
# Define the features and target
selected_features = ['global_radiation', 'humidity', 'precipitation', 'sunshine', 'wind_speed', 'temp_range']

# Prepare data
X = reshaped_df[selected_features]
y = reshaped_df['temp_mean']

# Initialize and fit RandomForestRegressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X, y)

# Initialize and fit GradientBoostingRegressor
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X, y)

# Initialize and fit XGBoost Regressor
xgb_model = XGBRegressor(random_state=42)
xgb_model.fit(X, y)

# Extract feature importances from each model
rf_importance = pd.Series(rf_model.feature_importances_, index=selected_features)
gb_importance = pd.Series(gb_model.feature_importances_, index=selected_features)
xgb_importance = pd.Series(xgb_model.feature_importances_, index=selected_features)

# Create a DataFrame to compare feature importance across models
feature_importance_df = pd.DataFrame({
    'Random Forest': rf_importance,
    'Gradient Boosting': gb_importance,
    'XGBoost': xgb_importance
})

# Sort the DataFrame by Random Forest importance for better comparison
feature_importance_df = feature_importance_df.sort_values(by='Random Forest', ascending=False)

# Print the comparison
print("Feature Importance Comparison Across Models:")
print(feature_importance_df)

# Plot the feature importances
import matplotlib.pyplot as plt
feature_importance_df.plot(kind='barh', figsize=(10, 6))
plt.title('Feature Importance Comparison Across Models')
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.show()

In [ ]:
reshaped_df.head()

In [ ]:
reshaped_df[selected_features].describe()

## Variable Encoding

In [ ]:
# Cyclic encoding for 'month'
reshaped_df['month_sin'] = np.sin(2 * np.pi * reshaped_df['month'] / 12)
reshaped_df['month_cos'] = np.cos(2 * np.pi * reshaped_df['month'] / 12)

#Cyclic encoding for 'day'
reshaped_df['day_sin'] = np.sin(2 * np.pi * reshaped_df['day'] / 31)
reshaped_df['day_cos'] = np.cos(2 * np.pi * reshaped_df['day'] / 31)

# One-hot encoding for 'city' column
reshaped_df = pd.get_dummies(reshaped_df, columns=['city'], drop_first=True)

# Drop 'datetime' column
reshaped_df.drop(columns=['datetime'], inplace=True)
reshaped_df.head()

## Modelling

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Function to evaluate model performance using cross-validation
def evaluate_model_cv(model, X, y, cv_folds=5):
    # Define custom scorers for RMSE, MAE, and R-squared
    scoring = {
        'R-squared': make_scorer(r2_score),
        'RMSE': make_scorer(mean_squared_error, squared=False),
        'MAE': make_scorer(mean_absolute_error)
    }
    
    # Initialize KFold cross-validation
    cv = KFold(n_splits=cv_folds, shuffle=True, random_state=42)

    # Perform cross-validation for each metric
    scores = {}
    for metric, scorer in scoring.items():
        score = cross_val_score(model, X, y, cv=cv, scoring=scorer)
        scores[metric] = np.mean(score)  # Average score across folds
    
    return scores

# Function to display model comparison and plot line graphs
def display_comparison(results):
    results_df = pd.DataFrame(results).T  # Transpose to get models as rows
    print(results_df)

    # Plot the comparison using line graphs for each metric
    fig, ax = plt.subplots(1, 3, figsize=(18, 6))
    
    # R-squared Plot
    results_df[['R-squared']].plot(ax=ax[0], marker='o', linestyle='-', color='blue')
    ax[0].set_title("R-squared Comparison")
    ax[0].set_ylabel("R-squared")
    ax[0].set_xlabel("Models")
    ax[0].set_xticks(range(len(results_df.index)))
    ax[0].set_xticklabels(results_df.index, rotation=45)
    
    # RMSE Plot
    results_df[['RMSE']].plot(ax=ax[1], marker='o', linestyle='-', color='red')
    ax[1].set_title("RMSE Comparison")
    ax[1].set_ylabel("RMSE")
    ax[1].set_xlabel("Models")
    ax[1].set_xticks(range(len(results_df.index)))
    ax[1].set_xticklabels(results_df.index, rotation=45)
    
    # MAE Plot
    results_df[['MAE']].plot(ax=ax[2], marker='o', linestyle='-', color='green')
    ax[2].set_title("MAE Comparison")
    ax[2].set_ylabel("MAE")
    ax[2].set_xlabel("Models")
    ax[2].set_xticks(range(len(results_df.index)))
    ax[2].set_xticklabels(results_df.index, rotation=45)

    plt.tight_layout()
    plt.show()

# Main function to run the comparison across multiple regressors with cross-validation
def compare_regressors_cv(X, y, cv_folds=5):
    # Initialize models
    models = {
        'Elastic Net': ElasticNet(random_state=42),
        'Random Forest': RandomForestRegressor(random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(random_state=42),
        'XGBoost': XGBRegressor(random_state=42),
        'LightGBM': LGBMRegressor(random_state=42),
        'CatBoost': CatBoostRegressor(random_state=42, verbose=0),
        'AdaBoost': AdaBoostRegressor(random_state=42)
    }

    # Dictionary to store results
    results = {}
    
    # Loop over models and store their performance
    for name, model in models.items():
        results[name] = evaluate_model_cv(model, X, y, cv_folds=cv_folds)
    
    # Display comparison of models
    display_comparison(results)

# Usage
# Assuming you already have reshaped_df, selected_features, and temp_mean as target
X = reshaped_df.drop(columns=['temp_mean'])  # Features
y = reshaped_df['temp_mean']  # Target variable

# Compare regressors with 5-fold cross-validation
compare_regressors_cv(X, y, cv_folds=5)

In [ ]:
import optuna
from catboost import CatBoostRegressor

# Objective function for CatBoost
def objective_catboost(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 10.0),
        'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_uniform('random_strength', 0.0, 1.0)
    }

    model = CatBoostRegressor(**param, random_state=42, verbose=0)
    
    # Perform a train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Calculate RMSE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    return rmse

# Create the study for CatBoost
study_catboost = optuna.create_study(direction='minimize')
study_catboost.optimize(objective_catboost, n_trials=50)

# Print the best hyperparameters and RMSE for CatBoost
print("Best hyperparameters for CatBoost:", study_catboost.best_trial.params)
print("Best RMSE for CatBoost:", study_catboost.best_trial.value)

In [ ]:
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Objective function for RandomForestRegressor
def objective_rf(trial):
    # Define the hyperparameters to tune
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 5, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])  # Replaced 'auto' with 'sqrt'
    
    # Initialize the RandomForestRegressor with trial hyperparameters
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )
    
    # Perform a train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Calculate RMSE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    return rmse

# Create the Optuna study to minimize the RMSE
study_rf = optuna.create_study(direction='minimize')
study_rf.optimize(objective_rf, n_trials=50)

# Print the best hyperparameters and RMSE
print("Best hyperparameters for Random Forest:", study_rf.best_trial.params)
print("Best RMSE for Random Forest:", study_rf.best_trial.value)

# Create the Optuna study to minimize the RMSE
study_rf = optuna.create_study(direction='minimize')
study_rf.optimize(objective_rf, n_trials=50)

# Print the best hyperparameters and RMSE
print("Best hyperparameters for Random Forest:", study_rf.best_trial.params)
print("Best RMSE for Random Forest:", study_rf.best_trial.value)

In [ ]:
import xgboost as xgb

# Objective function for XGBoost
def objective_xgb(trial):
    param = {
        'objective': 'reg:squarederror',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)
    }

    model = xgb.XGBRegressor(**param, random_state=42)
    
    # Perform a train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Calculate RMSE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    return rmse

# Create the study for XGBoost
study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=50)

# Print the best hyperparameters and RMSE for XGBoost
print("Best hyperparameters for XGBoost:", study_xgb.best_trial.params)
print("Best RMSE for XGBoost:", study_xgb.best_trial.value)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Final hyperparameters from Optuna
best_params = {
    'n_estimators': 984,
    'max_depth': 14,
    'learning_rate': 0.017,
    'subsample': 0.815,
    'colsample_bytree': 0.972,
    'random_state': 42
}

# Initialize XGBoost model with best hyperparameters
xgb_model = xgb.XGBRegressor(**best_params)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Calculate evaluation metrics
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print(f"Final Evaluation of XGBoost:")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

# Plot true vs predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5, color="blue")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color="red", lw=2)
plt.title("XGBoost: True vs Predicted Values")
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.grid(True)
plt.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Grouping by year and month, and calculating the mean for each group
grouped_test = X_test.groupby(['year', 'month'], as_index=False).mean()

# Select the city for which you want to visualize the data
city_to_plot = 'SONNBLICK'
city_test_mask = X_test['city_' + city_to_plot] == 1  # Mask to filter test data for this city

# Group y_test and y_pred by year and month similarly
y_test_grouped = pd.DataFrame({'Date': X_test[city_test_mask]['datetime'], 'mean_temp': y_test[city_test_mask]})
y_pred_grouped = pd.DataFrame({'Date': X_test[city_test_mask]['datetime'], 'mean_temp': y_pred[city_test_mask]})

# Now group them by year and month, taking mean temperature for both true and predicted values
y_test_grouped = y_test_grouped.groupby([y_test_grouped['Date'].dt.year, y_test_grouped['Date'].dt.month], as_index=False).mean()
y_pred_grouped = y_pred_grouped.groupby([y_pred_grouped['Date'].dt.year, y_pred_grouped['Date'].dt.month], as_index=False).mean()

# Plotly subplots
fig = make_subplots(rows=2, cols=1)

# Actual mean_temp (Truth)
fig.add_trace(go.Scatter(x=y_test_grouped['Date'], y=y_test_grouped['mean_temp'],
                         name='True Mean Temp', marker_color='LightSkyBlue'), row=1, col=1)

# Predicted mean_temp (Prediction)
fig.add_trace(go.Scatter(x=y_pred_grouped['Date'], y=y_pred_grouped['mean_temp'],
                         name='Predicted Mean Temp', marker_color='MediumPurple'), row=1, col=1)

# Plot on the second subplot for future predictions and comparison
fig.add_trace(go.Scatter(x=y_test_grouped['Date'], y=y_test_grouped['mean_temp'],
                         name='True Mean Temp (Future)', marker_color='LightSkyBlue', showlegend=False), row=2, col=1)

fig.add_trace(go.Scatter(x=y_pred_grouped['Date'], y=y_pred_grouped['mean_temp'],
                         name='Predicted Mean Temp (Future)', marker_color='MediumPurple', showlegend=False), row=2, col=1)

# Update layout
fig.update_layout(title=f"True vs Predicted Mean Temperature for {city_to_plot}",
                  xaxis_title="Date",
                  yaxis_title="Mean Temperature",
                  height=600, width=1000)

# Show plot
fig.show()

In [ ]:
reshaped_df.head()

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch.cuda.amp import GradScaler, autocast
import torch.nn.utils.spectral_norm as spectral_norm

# Define weather features and number of features
weather_features = ['global_radiation', 'humidity', 'precipitation', 'sunshine', 'wind_speed', 'temp_range']
n_features = len(weather_features)

# -------------------- 1. Network Definitions --------------------

# Generator network for multivariate time-series
class Generator(nn.Module):
    def __init__(self, latent_dim, seq_len, n_features):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, seq_len * n_features),
            nn.Tanh()
        )

    def forward(self, z):
        x = self.model(z)
        x = x.view(-1, seq_len, n_features)
        return x

# Discriminator network for multivariate time-series with Spectral Normalization
class Discriminator(nn.Module):
    def __init__(self, seq_len, n_features):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            spectral_norm(nn.Linear(seq_len * n_features, 512)),
            nn.LeakyReLU(0.2),
            spectral_norm(nn.Linear(512, 256)),
            nn.LeakyReLU(0.2),
            spectral_norm(nn.Linear(256, 1))
        )

    def forward(self, x):
        x = x.reshape(-1, seq_len * n_features)
        return self.model(x)

# -------------------- 2. Utility Functions --------------------

# Apply small Gaussian noise to discriminator inputs
def add_noise_to_data(data, noise_std=0.1):
    noise = noise_std * torch.randn_like(data)
    return data + noise

# LSGAN loss for Discriminator
def discriminator_loss(real_output, fake_output):
    real_loss = torch.mean((real_output - 1)**2)  # Least squares loss
    fake_loss = torch.mean(fake_output**2)
    return real_loss + fake_loss

# LSGAN loss for Generator
def generator_loss(fake_output):
    return torch.mean((fake_output - 1)**2)

# Data preprocessing using StandardScaler
def preprocess_data(data):
    data_scaler = StandardScaler()
    data = data.to_numpy()
    n_rows = data.shape[0]
    n_features = data.shape[1]
    truncated_rows = (n_rows // 24) * 24
    data = data[:truncated_rows, :]
    data = data_scaler.fit_transform(data.reshape(-1, n_features))
    data = data.reshape(-1, 24, n_features)
    return data, data_scaler

# Generate real data batches
def real_data_gen(weather_data, batch_size):
    idx = np.random.randint(0, len(weather_data) - batch_size)
    return torch.tensor(weather_data[idx:idx + batch_size], dtype=torch.float32)

# Generate random noise for the generator input
def generate_noise(batch_size, latent_dim):
    return torch.randn(batch_size, latent_dim)

# Function to train the discriminator
def train_discriminator(discriminator, optimizer, real_data, fake_data):
    optimizer.zero_grad()
    fake_data_detached = fake_data.detach()

    real_data_noisy = add_noise_to_data(real_data)
    fake_data_noisy = add_noise_to_data(fake_data_detached)

    real_output = discriminator(real_data_noisy)
    fake_output = discriminator(fake_data_noisy)
    
    d_loss = discriminator_loss(real_output, fake_output)
    return d_loss

# Function to train the generator
def train_generator(generator, discriminator, optimizer, fake_data):
    optimizer.zero_grad()
    prediction = discriminator(fake_data)
    g_loss = generator_loss(prediction)
    g_loss.backward()
    optimizer.step()
    return g_loss

# Visualize data with t-SNE
def visualize_tsne(real_data, synthetic_data):
    tsne = TSNE(n_components=2, perplexity=30)
    real_embedded = tsne.fit_transform(real_data)
    synthetic_embedded = tsne.fit_transform(synthetic_data)
    
    plt.scatter(real_embedded[:, 0], real_embedded[:, 1], label='Real', alpha=0.5)
    plt.scatter(synthetic_embedded[:, 0], real_embedded[:, 1], label='Synthetic', alpha=0.5)
    plt.legend()
    plt.title('t-SNE comparison of Real and Synthetic Data')
    plt.show()

# -------------------- 3. Training Setup --------------------

# Hyperparameters
seq_len = 24
latent_dim = 128
batch_size = 64
learning_rate_D = 5e-5  # Learning rate adjusted for LSGAN stability
learning_rate_G = 2e-4
num_epochs = 10000

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the generator and discriminator
generator = Generator(latent_dim, seq_len, n_features).to(device)
discriminator = Discriminator(seq_len, n_features).to(device)

# Optimizers
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate_D, betas=(0.5, 0.999))
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate_G, betas=(0.5, 0.999))

# Load and preprocess weather data
weather_data = reshaped_df[weather_features]
weather_data, data_scaler = preprocess_data(weather_data)

# -------------------- 4. Training Loop --------------------

d_losses = []
g_losses = []

for epoch in range(num_epochs):
    real_data = real_data_gen(weather_data, batch_size).to(device)

    # Generate new noise for each discriminator step
    noise = generate_noise(batch_size, latent_dim).to(device)
    fake_data = generator(noise)

    # Train the discriminator less frequently
    if epoch % 10 == 0:
        optimizer_D.zero_grad()
        d_error = train_discriminator(discriminator, optimizer_D, real_data, fake_data)
        d_error.backward()
        optimizer_D.step()

    # Train the generator more frequently
    optimizer_G.zero_grad()

    noise = generate_noise(batch_size, latent_dim).to(device)
    fake_data = generator(noise)

    g_error = train_generator(generator, discriminator, optimizer_G, fake_data)

    # Store losses
    d_losses.append(d_error.item() if 'd_error' in locals() else 0)
    g_losses.append(g_error.item())

    if epoch % 1000 == 0:
        print(f'Epoch {epoch} | D Loss: {d_error.item():.4f} | G Loss: {g_error.item():.4f}')
        
        # Visualize intermediate results with t-SNE
        with torch.no_grad():
            synthetic_data = generator(generate_noise(len(weather_data), latent_dim).to(device)).cpu().numpy().reshape(-1, n_features)
            visualize_tsne(weather_data.reshape(-1, n_features), synthetic_data)

# Plot the losses
plt.figure(figsize=(10, 5))
plt.plot(d_losses, label='Discriminator Loss')
plt.plot(g_losses, label='Generator Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curves')
plt.show()

In [ ]:
def plot_histograms(real_data, synthetic_data, feature_names):
    # Get the number of features based on the real data
    num_features = min(real_data.shape[1], synthetic_data.shape[1])  # Ensure we don't exceed the smallest dimension
    plt.figure(figsize=(15, 10))
    
    for i in range(num_features):
        plt.subplot(int(np.ceil(num_features / 3)), 3, i + 1)
        
        # Plot real data histogram
        plt.hist(real_data[:, i], bins=20, alpha=0.6, label='Real', density=True)
        
        # Plot synthetic data histogram
        plt.hist(synthetic_data[:, i], bins=20, alpha=0.6, label='Synthetic', density=True)
        
        # Ensure the title corresponds to the correct feature
        plt.title(feature_names[i] if i < len(feature_names) else f"Feature {i}")
        plt.legend()

    plt.tight_layout()
    plt.show()

# Assuming `weather_data` is a Pandas DataFrame and `synthetic_weather_data` is a NumPy array.
# Convert the real weather data (Pandas DataFrame) to a NumPy array
real_data = weather_data

# Ensure synthetic_weather_data is already a NumPy array (if not, convert it similarly)
# For demonstration purposes, I assume it is already a NumPy array.

# Plot the histograms
plot_histograms(real_data, synthetic_weather_data, weather_features)

In [ ]:
synthetic_weather_data